In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

import os
import sys
sys.path.insert(0,'../library')
from vexa.redis import get_redis,pop_items
from vexa.tools import log

AUDIO_API_PORT = os.getenv("AUDIO_API_PORT")
SERVICE_TOKEN = os.getenv("SERVICE_TOKEN")

In [7]:
import requests

url = "http://host.docker.internal:8002/get_segments"

params = {
        "service_token": SERVICE_TOKEN
    }



r = requests.get(url,params=params).json()
r

{'message': 'No more segments available'}